In [1]:
# 아이템 데이터
items = [['A', 'B', 'C'], ['A', 'B'], ['A', 'C'], ['B', 'C'],
        ['A', 'B', 'C'], ['B', 'C'], ['A', 'C'], ['A', 'B'],
        ['A'], ['A', 'C']]
print(items)

# 최소 지지도 임계치 설정
min_support = 0.3 #이 임계치 이상의 지지도를 가지는 아이템을 찾을 것

# 아이템 지지도 계산
item_counts = {}
transaction = len(items) # 전체 트랜잭션의 수
print(transaction)

# 각 트랜잭션에서 아이템의 지지도를 계산
for i in items:
#     print(i)
    for j in i:
#         print(j)
        if j in item_counts:
            item_counts[j] += 1
        else:
            item_counts[j] = 1

print(item_counts)

# 아이템 지지도가 최소 지지도 임계치 이상인 아이템들을 찾기
frequent = {}

for item, support in item_counts.items():
#     print(item, support)
    if support / transaction >= min_support:
        frequent[item] = support

print("아이템 지지도")
print(frequent)
    

[['A', 'B', 'C'], ['A', 'B'], ['A', 'C'], ['B', 'C'], ['A', 'B', 'C'], ['B', 'C'], ['A', 'C'], ['A', 'B'], ['A'], ['A', 'C']]
10
{'A': 8, 'B': 6, 'C': 7}
아이템 지지도
{'A': 8, 'B': 6, 'C': 7}


# apriori 알고리즘

In [2]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import pandas as pd

In [3]:
# 아이템 데이터
items = [['A', 'B', 'C'], ['A', 'B'], ['A', 'C'], ['B', 'C'],
        ['A', 'B', 'C'], ['B', 'C'], ['A', 'C'], ['A', 'B'],
        ['A'], ['A', 'C']]

df = pd.DataFrame(items, columns = ['Item1', 'Item2', 'Item3'])
df

,Item1,Item2,Item3
0,A,B,C
1,A,B,None
2,A,C,None
3,B,C,None
4,A,B,C
5,B,C,None
6,A,C,None
7,A,B,None
8,A,None,None
9,A,C,None


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Item1   10 non-null     object
 1   Item2   9 non-null      object
 2   Item3   2 non-null      object
dtypes: object(3)
memory usage: 372.0+ bytes


In [5]:
# 원핫인코딩
one_hot = pd.get_dummies(df)
one_hot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   Item1_A  10 non-null     bool 
 1   Item1_B  10 non-null     bool 
 2   Item2_B  10 non-null     bool 
 3   Item2_C  10 non-null     bool 
 4   Item3_C  10 non-null     bool 
dtypes: bool(5)
memory usage: 182.0 bytes


In [7]:
# 빈발항목 집합 찾기
frequent = apriori(one_hot, min_support = 0.3, use_colnames = True)
frequent
# one_hot : 분석할 데이터셋
# min_support : 최소 지지도(이 값보다 높은 지지도를 가진 항목 집합만 반환)
# use_colnames : 반환된 데이터프레임에서 항목 집합을 열이름으로 사용할 결정

,support,itemsets
0,0.8,(Item1_A)
1,0.4,(Item2_B)
2,0.5,(Item2_C)
3,0.4,"(Item1_A, Item2_B)"
4,0.3,"(Item1_A, Item2_C)"


In [9]:
# 연관규칙 추출
# 모든 연관규칙을 추출하려면 최소 임계치를 0으로 설정한다
rules = association_rules(frequent, metric = 'lift', min_threshold = 1.0)
rules

# frequent : 빈발항목집합 데이터프레임(apriori)
# metric : 규칙의 품질을 측정하는 평가지표
# lift, upport, confidence, leverage, confiction
# min_threshold : 최소 임계치(기본값 1.0)

# 데이터프레임 반환
# frequent_item_sets : 규칙의 전제항목
# antecedents : 규칙의 결과항목
# support : 규칙의 지지도를 나타냄
# confidence : 규칙의 신뢰도
# lift : 규칙의 리프트(독립적인 확률에 비해 규칙을 따를 확률의 상대적 증가)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Item1_A),(Item2_B),0.8,0.4,0.4,0.5,1.25,0.08,1.2,1.000000
1,(Item2_B),(Item1_A),0.4,0.8,0.4,1.0,1.25,0.08,inf,0.333333


In [10]:
rules['antecedents'] #조건절

0    (Item1_A)
1    (Item2_B)
Name: antecedents, dtype: object